In [18]:
import torch
import numpy as np
from sam2.sam2_image_predictor import SAM2ImagePredictor
from PIL import Image

predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-large")

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    img_path = "../../datasets/ml2hp/012/Right_Hand/Call/Vertical/images/001_left.bmp"
    img = Image.open(img_path).convert("RGBA") # Load the image
    predictor.set_image(img)

    masks, _, _ = predictor.predict(np.array([[img.width//2, img.height//2]]), np.array([1]))
    masks = np.transpose(masks, axes=[1, 2, 0])
    masks = (masks * 255).astype(np.uint8)

    print(type(masks), masks.shape)
    masks = Image.fromarray(masks).convert("L")
    img.putalpha(masks)
    img.save("masks.png")

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/rom42pla/.conda/envs/ml2hp/lib/python3.13/site-packages/torch/nn/modules/container.py", line 240, in forward
    def forward(self, input):
        for module in self:
            input = module(input)
                    ~~~~~~ <--- HERE
        return input
  File "/home/rom42pla/.conda/envs/ml2hp/lib/python3.13/site-packages/torchvision/transforms/transforms.py", line 277, in forward
            Tensor: Normalized Tensor image.
        """
        return F.normalize(tensor, self.mean, self.std, self.inplace)
               ~~~~~~~~~~~ <--- HERE
  File "/home/rom42pla/.conda/envs/ml2hp/lib/python3.13/site-packages/torchvision/transforms/functional.py", line 350, in normalize
        raise TypeError(f"img should be Tensor Image. Got {type(tensor)}")

    return F_t.normalize(tensor, mean=mean, std=std, inplace=inplace)
           ~~~~~~~~~~~~~ <--